In [1]:
from sympy import *
import numpy as np

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

This is the derivation without inertia of the rod. For "correct equations": https://coneural.org/florian/papers/05_cart_pole.pdf
![Alt text](pics/diagram.jpg)

In [2]:
#Symbols
t = symbols('t')
    #states
x = Function('x')(t)
θ = Function('θ')(t)
    #coordinates of cart and polemass
x1, y1, x2, y2 = symbols('x1 y1 x2 y2')
    #length of pole, mass of cart, mass of pole, gravity
l, m1, m2, g = symbols('l m1 m2 g')

#Visuals
    #use clean() to render d/dt as dots and remove the (t) in functions
x_d_disp = Function('\dot{x}')(t)
θ_d_disp = Function('\dot{θ}')(t)
x_dd_disp = Function('\ddot{x}')(t)
θ_dd_disp = Function('\ddot{θ}')(t)
dots = {x.diff(t):x_d_disp, θ.diff(t):θ_d_disp, x_d_disp.diff(t):x_dd_disp, θ_d_disp.diff(t):θ_dd_disp}
no_t = { 
    x:symbols('x'), θ:symbols('θ'), 
    x_d_disp:symbols('\dot{x}'), θ_d_disp:symbols('\dot{θ}'), 
    x_dd_disp:symbols('\ddot{x}'), θ_dd_disp:symbols('\ddot{θ}'), 
}
clean = lambda expr : expr.subs(dots).subs(dots).subs(no_t)
    #keep as fraction rather than floating point
half = Rational(1,2)

#Positions
x1 = x
y1 = 0
x2 = x1 + l*cos(θ)
y2 = l*sin(θ)

#Kinetic energy
T = half*m1*diff(x1,t)**2 + half*m2*(diff(x2,t)**2 + diff(y2,t)**2)
T = trigsimp(T)

#Potential energy
V = m2*g*l*sin(θ)

#Lagrangian
L = T - V

clean(L)

\dot{x}**2*m1/2 - g*l*m2*sin(θ) + m2*(\dot{x}**2 - 2*\dot{x}*\dot{θ}*l*sin(θ) + \dot{θ}**2*l**2)/2

In [3]:
#Euler-Lagrange
Fx, Fθ = symbols('F_x, F_θ')
Eq_x = Eq(diff(diff(L, diff(x)), t) - diff(L, x), Fx)
Eq_θ = Eq(diff(diff(L, diff(θ)), t) - diff(L, θ), Fθ)

clean(Matrix([
    Eq_x.simplify(), 
    Eq_θ.simplify()
]))

Matrix([
[Eq(F_x, \ddot{x}*m1 - m2*(-\ddot{x} + \ddot{θ}*l*sin(θ) + \dot{θ}**2*l*cos(θ)))],
[                       Eq(F_θ, l*m2*(-\ddot{x}*sin(θ) + \ddot{θ}*l + g*cos(θ)))]])

In [4]:
second_derivatives = solve([Eq_x, Eq_θ], [diff(diff(x)), diff(diff(θ))])
x_dd = second_derivatives[diff(diff(x))].simplify()
θ_dd = second_derivatives[diff(diff(θ))].simplify()

In [5]:
clean(Eq(x_dd_disp, x_dd))

Eq(\ddot{x}, (F_x*l + F_θ*sin(θ) + \dot{θ}**2*l**2*m2*cos(θ) - g*l*m2*sin(2*θ)/2)/(l*(m1 + m2*cos(θ)**2)))

In [6]:
clean(Eq(θ_dd_disp, θ_dd))

Eq(\ddot{θ}, (F_x*l*m2*sin(θ) + F_θ*m1 + F_θ*m2 + \dot{θ}**2*l**2*m2**2*sin(2*θ)/2 - g*l*m1*m2*cos(θ) - g*l*m2**2*cos(θ))/(l**2*m2*(m1 + m2*cos(θ)**2)))

In [18]:
x_dd_noFθ = x_dd.subs(Fθ, 0).simplify()
clean(Eq(x_dd_disp, x_dd_noFθ))

Eq(\ddot{x}, (F_x + \dot{θ}**2*l*m2*cos(θ) - g*m2*sin(2*θ)/2)/(m1 + m2*cos(θ)**2))

In [19]:
θ_dd_noFθ = θ_dd.subs(Fθ, 0).simplify()
clean(Eq(θ_dd_disp, θ_dd_noFθ))

Eq(\ddot{θ}, (F_x*sin(θ) + \dot{θ}**2*l*m2*sin(2*θ)/2 - g*m1*cos(θ) - g*m2*cos(θ))/(l*(m1 + m2*cos(θ)**2)))

check work: https://danielpiedrahita.wordpress.com/portfolio/cart-pole-control/
![Alt text](pics/no_inertia.png)

In [25]:
#text format
print('x_dd =', x_dd_noFθ)
print('theta_dd =', θ_dd_noFθ)

x_dd = (F_x - g*m2*sin(2*θ(t))/2 + l*m2*cos(θ(t))*Derivative(θ(t), t)**2)/(m1 + m2*cos(θ(t))**2)
theta_dd = (F_x*sin(θ(t)) - g*m1*cos(θ(t)) - g*m2*cos(θ(t)) + l*m2*sin(2*θ(t))*Derivative(θ(t), t)**2/2)/(l*(m1 + m2*cos(θ(t))**2))
